In [ ]:
import pandas as pd

In [ ]:
df_bg_gold = pd.read_csv('gold_standard_nested_links_diagnosis_2023.csv', header=0)
df_bg_gold.head(50)

In [ ]:
def replace_label(label):
    if 'B-' in label:
        return 'B-MEDTERM'
    elif 'I-' in label:
        return 'I-MEDTERM'
    return 'O'

In [ ]:
prev_doc_id = 0
prev_label = 'O'
with open("bg_conll_30_test_2023.txt", "w") as file1:  
    for index, row in df_bg_gold.iterrows():
        if prev_doc_id != row['doc_id']:
            file1.write('\n')

        label = str(row['terms'])
        if label == '':
            label = 'O'
        #label = row['type0']
        #if label == 'O':
        #    label = row['type1']
        #if label == 'O':
        #    label = row['type2']

        label = replace_label(label)
        #if 'MEDTERM' in prev_label and 'B-' in label:
        #    label = 'I-MEDTERM'
        #print(f"{row['text']} {label}")
        file1.write(f"{row['text']} {label}\n")
        
        prev_doc_id = row['doc_id']
        prev_label = label

In [ ]:
#filename	mark	label	off0	off1	span	code	semantic_rel
entities = []
current_entity = {
    'filename': '',
    'span': '',
    'off0': -1,
    'off1': -1    
}
current_start = 0
current_doc = ''
for index, row in df_bg_gold.iterrows():    
    label = str(row['terms'])
    
    if current_doc == '' or current_doc != row['doc_id']:
        current_doc = row['doc_id']
        current_start = 0
    
    if label.startswith('B-'):
        if len(current_entity['span']) > 0:
            entities.append(current_entity)
            current_entity = {
                'filename': row['doc_id'],
                'span': '',
                'off0': -1,
                'off1': -1    
            }
            
        current_entity = {
            'filename': row['doc_id'],
            'span': row['text'],
            'off0': current_start,
            'off1': current_start + len(row['text'])    
        }
        
    elif label.startswith('I-'):
        current_entity['span'] += ' ' + row['text']
        current_entity['off1'] += len(row['text']) + 1
    else:
        if len(current_entity['span']) > 0:
            entities.append(current_entity)
            current_entity = {
                'filename': row['doc_id'],
                'span': '',
                'off0': -1,
                'off1': -1    
            }
    
    current_start += len(row['text']) + 1
    print(f"'{row['text']}':{current_start}")

In [ ]:
entities

In [ ]:
df_entities = pd.DataFrame(entities)
df_entities.head()

In [ ]:
df_entities.to_csv('bg_gold_standard_entities.tsv', sep='\t', index=False)